# Example 2: Simple 2D cell signaling model

We model a reaction between the cell interior and cell membrane in a 2D geometry:
- Cyto - 2D cell "volume"
- PM - 1D cell boundary (represents plasma membrane)

Model from [Rangamani et al, 2013, Cell](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3874130/). A cytosolic species, "A", reacts with a species on the PM, "B", to form a new species on the PM, "X". The resulting PDE and boundary condition for species A are as follows:

$$
\frac{\partial{C_A}}{\partial{t}} = D_A \nabla ^2 C_A \quad \text{in} \; \Omega_{Cyto}\\
\text{B.C. for A:} \quad D_A (\textbf{n} \cdot \nabla C_A)  = -k_{on} C_A N_X + k_{off} N_B \quad \text{on} \; \Gamma_{PM}
$$

Similarly, the PDEs for X and B are given by:
$$
\frac{\partial{N_X}}{\partial{t}} = D_X \nabla ^2 N_X - k_{on} C_A N_X + k_{off} N_B \quad \text{on} \; \Gamma_{PM}\\
\frac{\partial{N_B}}{\partial{t}} = D_B \nabla ^2 N_B + k_{on} C_A N_X - k_{off} N_B \quad \text{on} \; \Gamma_{PM}
$$

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
img_A = mpimg.imread('axb-diagram.png')
plt.imshow(img_A)
plt.axis('off')

Imports and logger initialization:

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import gmsh  # must be imported before pyvista if dolfin is imported first

from smart import config, common, mesh, model, mesh_tools, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

First, we define the various units for use in the model.

In [ ]:
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
D_unit = um**2 / sec
surf_unit = molecule / um**2
flux_unit = molecule / (um * sec)
edge_unit = molecule / um

Next we generate the model by assembling the compartment, species, parameter, and reaction containers (see Example 1 or API documentation for more details).

In [ ]:
# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
Cyto = Compartment("Cyto", 2, um, 1)
PM = Compartment("PM", 1, um, 3)
cc = CompartmentContainer()
cc.add([Cyto, PM])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
A = Species("A", 1.0, surf_unit, 100.0, D_unit, "Cyto")
X = Species("X", 1.0, edge_unit, 1.0, D_unit, "PM")
B = Species("B", 0.0, edge_unit, 1.0, D_unit, "PM")
sc = SpeciesContainer()
sc.add([A, X, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================

# Reaction of A and X to make B (Cyto-PM reaction)
kon = Parameter("kon", 1.0, 1/(surf_unit*sec))
koff = Parameter("koff", 1.0, 1/sec)
r1 = Reaction("r1", ["A", "X"], ["B"],
              param_map={"on": "kon", "off": "koff"},
              species_map={"A": "A", "X": "X", "B": "B"})

pc = ParameterContainer()
pc.add([kon, koff])
rc = ReactionContainer()
rc.add([r1])

Now we create a circular mesh (mesh built using gmsh in `smart.mesh_tools`), along with marker functions `mf2` and `mf1`.

In [ ]:
# Create mesh
h_ellipse = 0.1
xrad = 1.0
yrad = 1.0
surf_tag = 1
edge_tag = 3
ellipse_mesh, mf1, mf2 = mesh_tools.create_ellipses(xrad, yrad, hEdge=h_ellipse,
                                                    outer_tag=surf_tag, outer_marker=edge_tag)
visualization.plot_dolfin_mesh(ellipse_mesh, mf2, view_xy=True)

Write mesh and meshfunctions to file, then create `mesh.ParentMesh` object.

In [ ]:
mesh_folder = pathlib.Path("ellipse_mesh_AR1")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "ellipse_mesh.h5"
mesh_tools.write_mesh(ellipse_mesh, mf1, mf2, mesh_file)

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
)

Initialize model and solvers.

In [ ]:
config_cur = config.Config()
config_cur.solver.update(
    {
        "final_t": 10.0,
        "initial_dt": 0.05,
        "time_precision": 6,
    }
)

model_cur = model.Model(pc, sc, cc, rc, config_cur, parent_mesh)
model_cur.initialize()

Save model information to .pkl file and write initial conditions to file.

In [ ]:
model_cur.to_pickle('model_cur.pkl')
results = dict()
result_folder = pathlib.Path("resultsEllipse_AR1_loaded")
result_folder.mkdir(exist_ok=True)
for species_name, species in model_cur.sc.items:
    results[species_name] = d.XDMFFile(
        model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)

Solve the system until `model_cur.t > model_cur.final_t`.

In [ ]:
tvec = [0]
avg_A = [A.initial_condition]
avg_X = [X.initial_condition]
avg_B = [B.initial_condition]
# Set loglevel to warning in order not to pollute notebook output
logger.setLevel(logging.WARNING)

while True:
    # Solve the system
    model_cur.monolithic_solve()
    # Save results for post processing
    for species_name, species in model_cur.sc.items:
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    dx = d.Measure("dx", domain=model_cur.cc['Cyto'].dolfin_mesh)
    int_val = d.assemble_mixed(model_cur.sc['A'].u['u']*dx)
    volume = d.assemble_mixed(1.0*dx)
    avg_A.append(int_val / volume)
    avg_X.append(d.assemble_mixed(X.u["u"]*d.Measure("dx",PM.dolfin_mesh)) 
                 / (d.assemble_mixed(1.0*d.Measure("dx",PM.dolfin_mesh))))
    avg_B.append(d.assemble_mixed(B.u["u"]*d.Measure("dx",PM.dolfin_mesh)) 
                 / (d.assemble_mixed(1.0*d.Measure("dx",PM.dolfin_mesh))))
    tvec.append(model_cur.t)
    # End if we've passed the final time
    if model_cur.t >= model_cur.final_t:
        break

Now we plot the concentration of A in the cell over time and compare this to analytical predictions for a high value of the diffusion coefficient. As $D_A \rightarrow \infty$, the steady state concentration of A will be given by the positive root of the following polynomial:

$$
-k_{on} c_A ^2 - \left( k_{on} c_{X} (t=0) \frac{SA_{PM}}{vol_{cyto}} + k_{off} - k_{on} c_A (t=0)   \right) c_A + k_{off} c_A(t=0)
$$

Note that in this 2D case, $SA_{PM}$ is the perimeter of the ellipse and $vol_{cyto}$ is the area of the ellipse. These can be thought of as a surface area and volume if we extrude the 2D shape by some characteristic thickness.

In [ ]:
plt.plot(tvec, avg_A, label='SMART simulation')
plt.xlabel('Time (s)')
plt.ylabel('A concentration $\mathrm{(molecules/μm^2)}$')
SA_vol = 4/(xrad + yrad)
root_vals = np.roots([-kon.value,
                      -kon.value*X.initial_condition*SA_vol - koff.value + kon.value*A.initial_condition,
                      koff.value*A.initial_condition])
ss_pred = root_vals[root_vals > 0]
plt.plot(tvec, np.ones(len(avg_A))*ss_pred, '--', label='Steady-state analytical prediction')
plt.legend()
percent_error_analytical = 100*np.abs(ss_pred-avg_A[-1])/ss_pred
assert percent_error_analytical < 0.1,\
    f"Failed test: Example 2 results deviate {percent_error_analytical:.3f}% from the analytical prediction"

Plot A concentration in the cell at the final time point.

In [ ]:
visualization.plot(model_cur.sc["A"].u["u"], view_xy=True)
# also save to file for comparison visualization in the end
meshimg_folder = pathlib.Path("mesh_images")
meshimg_folder = meshimg_folder.resolve()
meshimg_folder.mkdir(exist_ok=True)
meshimg_file = meshimg_folder / "ellipse_mesh_AR1.png"

# Put all images in a dictionary for easy recovery later
images = {}
images[1] = visualization.plot(model_cur.sc["A"].u["u"], view_xy=True, filename=meshimg_file)

Now iterate over three different cell aspect ratios and compare dynamics between cases.

In [ ]:
# initialize plot to compare dynamics
plt.plot(tvec, avg_A, label='Aspect ratio = 1')
plt.xlabel('Time (s)')
plt.ylabel('A concentration $\mathrm{(molecules/μm^2)}$')
# iterate over additional aspect ratios, enforcing the same ellipsoid area in all cases
aspect_ratios = [1, 1.5**2, 4, 16]
ss_calc = [avg_A[-1]]
for i in range(1, len(aspect_ratios)):
    # Create mesh
    xrad = 1.0*np.sqrt(aspect_ratios[i])
    yrad = 1.0/np.sqrt(aspect_ratios[i])
    ellipse_mesh, mf1, mf2 = mesh_tools.create_ellipses(xrad, yrad, hEdge=h_ellipse,
                                                        outer_tag=surf_tag, outer_marker=edge_tag)
    mesh_folder = pathlib.Path(f"ellipse_mesh_AR{aspect_ratios[i]}")
    mesh_folder.mkdir(exist_ok=True)
    mesh_tools.write_mesh(ellipse_mesh, mf1, mf2, mesh_file)
    parent_mesh = mesh.ParentMesh(
        mesh_filename=str(mesh_file),
        mesh_filetype="hdf5",
        name="parent_mesh",
    )
    config_cur = config.Config()
    config_cur.solver.update(
        {
            "final_t": 5.0,
            "initial_dt": 0.05,
            "time_precision": 6,
        }
    )
    model_cur = model.Model(pc, sc, cc, rc, config_cur, parent_mesh)
    model_cur.initialize()
    results = dict()
    result_folder = pathlib.Path(f"resultsEllipse_AR{aspect_ratios[i]}")
    result_folder.mkdir(exist_ok=True)
    for species_name, species in model_cur.sc.items:
        results[species_name] = d.XDMFFile(
            model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    avg_A = [A.initial_condition]
    dx = d.Measure("dx", domain=model_cur.cc['Cyto'].dolfin_mesh)
    volume = d.assemble_mixed(1.0*dx)
    while True:
        # Solve the system
        model_cur.monolithic_solve()
        # Save results for post processing
        for species_name, species in model_cur.sc.items:
            results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
        int_val = d.assemble_mixed(model_cur.sc['A'].u['u']*dx)
        avg_A.append(int_val / volume)
        # End if we've passed the final time
        if model_cur.t >= model_cur.final_t:
            break
    plt.plot(model_cur.tvec, avg_A, label=f"Aspect ratio = {aspect_ratios[i]}")
    ss_calc.append(avg_A[-1])
    meshimg_file = meshimg_folder / f"ellipse_mesh_AR{aspect_ratios[i]}.png"
    images[aspect_ratios[i]] = visualization.plot(model_cur.sc["A"].u["u"], view_xy=True, filename=meshimg_file)
plt.legend()

Compare steady-state values with those from past runs using SMART (regression test):

In [ ]:
ss_stored = [0.41384929369203755, 0.38021621231324093, 0.32689497127690803, 0.18803994874948224]
percent_error = max(np.abs(np.array(ss_stored) - np.array(ss_calc))/np.array(ss_stored))*100
assert percent_error < .01,\
    f"Failed regression test: Example 2 results deviate {percent_error:.3f}% from the previous numerical solution"

Now, display the final states from all simulations for direct comparison.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 15))
for axi, (aspect_ratio, image) in zip(ax.flatten(), images.items()):
    axi.imshow(image)
    axi.axis('off')
    axi.set_title(f"Aspect ratio = {aspect_ratio}")